In [4]:
import numpy as np
from docx import Document
from docx.shared import Pt

In [5]:
f='gstr1'
fname=f'{f}.docx'

doc=Document(fname)
all_tables=[]
for table in doc.tables:
    rows=len(table.rows)
    cols=len(table.columns)
    data=np.empty((rows,cols), dtype=object)
    for i,row in enumerate(table.rows):
        for j,cell in enumerate(row.cells):
            data[i,j]=cell.text.strip()
    all_tables.append(data)

In [6]:
def update_document(row, col, new_value, table_index):
    if row >= len(doc.tables[table_index].rows) or col >= len(doc.tables[table_index].columns):
        print("Warning: The specified cell does not exist in the table.")
        return    
    all_tables[table_index][row, col] = new_value
    cell = doc.tables[table_index].rows[row].cells[col]
    cell.text = str(new_value)
    for paragraph in cell.paragraphs:
        for run in paragraph.runs:
            run.font.name, run.font.size = 'Roboto', Pt(7.5)

In [7]:
def fn(x):
    if x[-1][0][0].isdigit():
        return 'yes'
    return 'no'

for i in range(len(all_tables)):
    print(f'all_tables[{i}]',fn(all_tables[i]))

all_tables[0] no
all_tables[1] yes
all_tables[2] yes
all_tables[3] yes
all_tables[4] no
all_tables[5] no


In [8]:
def fn(x,y,k=0):
    if k==1:
        c=x[-4]
    else:
        c=x[-1]
    c_reshaped=c.reshape(1, -1)
    return np.vstack((c_reshaped, y))

all_tables[3]=fn(all_tables[2],all_tables[3])
all_tables[4]=fn(all_tables[3],all_tables[4])
all_tables[5]=fn(all_tables[4],all_tables[5],1)

In [9]:
def get_desc(x):
    l=[]
    for row_index, row in enumerate(x):
        try:
            z=int(row[0][0])
        except:
            z=-1
        if(z!=-1):
            l.append(row_index)
    return l

t=[]
for i in range(len(all_tables)):
    t.append(get_desc(all_tables[i]))
t

[[],
 [0, 1, 2, 3, 4],
 [1, 4, 7, 10, 15, 20, 23],
 [0, 4, 10, 14, 18, 22, 28, 34, 38],
 [0, 12, 19, 31],
 [0, 6, 10, 13, 16, 20, 24, 27]]

In [10]:
lists=[]
for i in range(2):
    for row_index, row in enumerate(all_tables[i]):
        for col_index, value in enumerate(row):
            if len(value)==0:
                d={}
                d['Row']=all_tables[i][row_index][-2]
                d['Value']='NaN'
                d['Column']=''
                d['Coordinates']=(row_index,col_index,i)
                lists.append(d)

In [11]:
for g in range(2,len(all_tables)):
    x=all_tables[g]
    l=t[g]
    for row_index, row in enumerate(x):
        for col_index, value in enumerate(row):
            try:
                x1=int(value.strip())
            except:
                x1=-1
            if x1!=-1 or value=='0.00':
                d={}
                d['Description']=list(set(x[max([num for num in l if num<row_index])]))[0]
                d['Row']=x[row_index,0]
                d['Value']='NaN'
                if g==2:
                    d['Coordinates']=(row_index,col_index,g)
                    d['Column']=x[0,col_index]
                else:
                    d['Coordinates']=(row_index-1,col_index,g)
                    d['Column']=x[1,col_index]
                lists.append(d)

for i in range(len(lists)-5, len(lists)):
    del lists[i]['Description']

In [12]:
# import pickle
# with open('data_params_2.pkl', 'wb') as f:
#     pickle.dump(lists, f)

# with open('data_params.pkl', 'rb') as f:
#     lists=pickle.load(f)

In [13]:
o={}
o['Value (₹)']='Value'
o['Integrated Tax (₹)']='IGST'
o['Central Tax (₹)']='CGST'
o['State/UT Tax (₹)']='SGST'
o['Cess (₹)']='Cess'
o['']=''

k=[f]
for i_index, i in enumerate(lists):
    try:
        s=' | '.join([f"{i_index}.",i['Row'],o[i['Column']],i['Description']])
    except:
        s=' | '.join([f"{i_index}.",i['Row'],o[i['Column']],''])
    k.append(s)
k

['gstr1',
 '0. | Financial year |  | ',
 '1. | Tax period |  | ',
 '2. | GSTIN |  | ',
 '3. | Legal name of the registered person |  | ',
 '4. | Trade name if any |  | ',
 '5. | ARN |  | ',
 '6. | ARN date |  | ',
 '7. | Total | Value | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular',
 '8. | Total | IGST | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular',
 '9. | Total | CGST | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular',
 '10. | Total | SGST | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular',
 '11. | Total | Cess | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular',
 '12. | Total | Value | 4B - Taxable outward supplies made to registered persons attracting tax on reverse charge - B2B Rev

In [14]:
import pickle
with open('gstr1_params.pkl', 'wb') as f:
    pickle.dump(k, f)

In [15]:
# doc.save(f"{f}_Updated.docx")

In [16]:
# for i in lists:
#     update_document(i['Coordinates'][0], i['Coordinates'][1], i['Coordinates'], i['Coordinates'][2])

In [17]:
# import docx2pdf

In [18]:
# docx2pdf.convert(f"{f}_Updated.docx", f"{f}.pdf")